# Diverse Dataset

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1CmXjXVrmPtpAVBaogBSuDclM0O6Zzewf?usp=sharing)

The purpose of this notebook is to download the DIVERSE dataset and convert it into a format that can be used for training the OpenAssistant.

The DIVERSE repo can be found here: https://github.com/microsoft/CodeT/tree/main/DIVERSE

If you extend or use this work, please cite the relevant papers:
```
@article{li2022advance,
  title={On the Advance of Making Language Models Better Reasoners},
  author={Li, Yifei and Lin, Zeqi and Zhang, Shizhuo and Fu, Qiang and Chen, Bei and Lou, Jian-Guang and Chen, Weizhu},
  journal={arXiv preprint arXiv:2206.02336},
  year={2022}
}

```

# OpenAssistant Data Scheme

We will use the data scheme that can be found in the docs for Open-Assistant. This code is taken from the StackExchange notebook.

In [13]:
from typing import TypeVar, List, Dict, Any, Literal
from json import JSONEncoder

T = TypeVar("T", bound="ConversationTreeNode")


class ConversationTreeNode:
    text: str  # The text of the node
    role: Literal["prompter", "assistant"]  # Whether the node is a user prompt/follow-up or an assistant response
    children: List[T]  # The children of the node (if you have a linear conversation, this will be of length 0 or 1)
    metadata: Dict[str, Any]  # Node metadata (see below)

    def __init__(
        self, text: str, role: Literal["prompter", "assistant"], children: List[T], metadata: Dict[str, Any]
    ) -> None:
        self.text = text
        self.role = role
        self.children = children
        self.metadata = metadata


class ConversationTree:
    root: ConversationTreeNode  # The node containing the initial prompt
    metadata: Dict[str, Any]  # Tree metadata, different from root node metadata.

    def __init__(self, root: ConversationTreeNode, metadata: Dict[str, Any]) -> None:
        self.root = root
        self.metadata = metadata


# subclass JSONEncoder
class TreeEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__

# Download and convert

We firstly import pandas and any other libraries that we'll need.

In [14]:
import pandas as pd
import json

The following is a simple function to take the data (which has two columns) and convert it to a tree with a root note (question) and one child (answer).

In [19]:
import re


def convert_diverse(dataset_json_path):
    # read files using pandas
    ds = pd.read_json(dataset_json_path, lines=True)

    # create dataset name from path
    ds_name = "diverse" + file.split("data")[-1].replace("/", "_").split(".")[0]
    print("*****", ds_name, "****")
    print("Example of raw dataset")
    print(ds.head(2))

    # create conversation forest
    # Print first sample so the user of this notebook has an idea of what he's looking at
    first_sample = True
    print("\nExamples from converted dataset")
    conversation_forest = []
    for item in ds["context"]:
        # build nodes and tree
        # Find all answers:

        answers = re.findall(r"Answer:?(.*?)#", item.replace("\n", " "))
        questions = re.findall(r"Question:?(.*?) Answer:", item.replace("\n", " "))

        # The last question does not contain an aswer so we drop it every time.
        if len(answers) < len(questions):
            questions.pop(-1)

        for answer, question in zip(answers, questions):
            if first_sample:
                print(f"Q: {question}")
                print(f"A: {answer}")
            root = ConversationTreeNode(text=question, role="prompter", children=[], metadata=None)
            child = ConversationTreeNode(text=answer, role="assistant", children=[], metadata=None)
            root.children.append(child)
            conversation_tree = ConversationTree(root=root, metadata={"dataset": ds_name})
            conversation_forest.append(conversation_tree)

        first_sample = False

    conversation_forest_json = [
        json.loads(TreeEncoder().encode(conversation_tree)) for conversation_tree in conversation_forest
    ]

    print(json.dumps(conversation_forest_json, indent=4), file=open(f"./{ds_name}.json", "w+"))
    print("\n")

We now clone the repository containing the dataset

In [16]:
!git clone https://github.com/microsoft/CodeT.git

Cloning into 'CodeT'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 144 (delta 1), reused 0 (delta 0), pack-reused 128ving objects:   8% (12/144), 3.70 MiB | 1.35 MiB/s   Receiving objects:  12% (18/144), 5.13 MiB | 1.58 MiB/s   Receiving objects:  13% (19/144), 11.36 MiB | 2.39 MiB/s   Receiving objects:  13% (20/144), 19.19 MiB | 3.97 MiB/s   Receiving objects:  15% (22/144), 19.19 MiB | 3.97 MiB/s   Receiving objects:  23% (34/144), 22.15 MiB | 4.37 MiB/s   Receiving objects:  27% (39/144), 22.15 MiB | 4.37 MiB/s   Receiving objects:  29% (42/144), 25.30 MiB | 4.77 MiB/s   Receiving objects:  32% (47/144), 28.71 MiB | 5.22 MiB/s   Receiving objects:  41% (60/144), 32.41 MiB | 5.62 MiB/s   Receiving objects:  54% (78/144), 32.41 MiB | 5.62 MiB/s   Receiving objects:  60% (87/144), 39.34 MiB | 6.20 MiB/s   Receiving objects:  61% (88/144), 47.14 MiB | 6.79 MiB/s   Receiving objects

In [17]:
diverse_files = [
    "CodeT/DIVERSE/data/sqa/split1/test.jsonl",
    "CodeT/DIVERSE/data/sqa/split1/train.jsonl",
    "CodeT/DIVERSE/data/sqa/split2/test.jsonl",
    "CodeT/DIVERSE/data/sqa/split2/train.jsonl",
    "CodeT/DIVERSE/data/gsm8k/test.jsonl",
    "CodeT/DIVERSE/data/gsm8k/train.jsonl",
]

In [20]:
for file in diverse_files:
    convert_diverse(file)

***** diverse_sqa_split1_test ****
Example of raw dataset
                                             context  \
0  Question:\nIs clerk of Supreme Court of Canada...   
1  Question:\nIs Saturn named after king of gods ...   

                                             samples  \
0  [Snoopy is a dog.\nChance is a dog.\nDogs look...   
1  [Snoopy is a cartoon dog.\nChance is a cartoon...   

                                            metadata  
0  {'type': 'solution', 'question': 'Does Snoopy ...  
1  {'type': 'solution', 'question': 'Does Snoopy ...  

Examples from converted dataset
Q:  Is clerk of Supreme Court of Canada safe profession for someone with seismophobia?
A:  The Supreme Court of Canada is in Ottawa, Canada. Ottawa is in Ontario, Canada. Ontario is in the Canadian Shield. The Canadian Shield is a stable tectonic plate. Thus, Ottawa is not prone to earthquakes. Thus, the clerk of the Supreme Court of Canada is a safe profession for someone with seismophobia. So the answ